# Opening images in Keras
Todo:
 - Change from Pillow to rasterio to read image
 - 

Sources:
    - https://medium.com/@ensembledme/writing-custom-keras-generators-fe815d992c5a
    - https://stackoverflow.com/questions/46493419/use-a-generator-for-keras-model-fit-generator
    - https://stackoverflow.com/questions/50195967/how-can-i-create-a-4d-numpy-array-from-images
    - https://www.google.com/search?client=firefox-b-ab&q=keras+make+image+generator
 

In [7]:
from PIL import Image

import keras
from PIL import Image

import imageio
import random
import scipy
import numpy as np

import rasterio
import numpy as np


## Variables

In [224]:
image_dir_path = '../Sorted_tiles/'       # Path to images

## Read imagelabels

In [50]:
import pandas as pd
labelfile = pd.read_csv('../Qgis/labeled_tiles.csv')
labelfile.head()

,ID,Tile_name,LIVSM1,LIVSM2,mult_bin_labels,1,2,3,4,5,6,7,8,9,10,11,12,99
0,0,all_bands_merged_01_31.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,all_bands_merged_01_32.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,all_bands_merged_01_33.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
3,3,all_bands_merged_01_34.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1
4,4,all_bands_merged_01_35.tif,99,99,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]",0,0,0,0,0,0,0,0,0,0,0,0,1


In [11]:
labelfile.describe()

,ID,LIVSM1,LIVSM2,1,2,3,4,5,6,7,8,9,10,11,12,99
count,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.000000,551.0,551.0,551.0,551.0,551.0,551.0,551.0,551.000000
mean,275.000000,92.520871,92.459165,0.025408,0.021779,0.016334,0.014519,0.019964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.932849
std,159.204271,24.174724,24.405321,0.157505,0.146092,0.126871,0.119726,0.140003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250510
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
25%,137.500000,99.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
50%,275.000000,99.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
75%,412.500000,99.000000,99.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
max,550.000000,99.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000


### Balance classes

In [220]:
# Stratified sample of the classes to get a balanced dataset with the same number of classes to train on.
# https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/
# https://stackoverflow.com/questions/44114463/stratified-sampling-in-pandas

# Maximum number of elements in each class. If the classe have less than max every sample is included.
clasnumber_of_samples = 12

labelfile_balanced = labelfile.groupby(['LIVSM1'], group_keys=False).apply(lambda x: x.sample(min(len(x), clasnumber_of_samples))).reset_index(drop=True)
labelfile_balanced.head()

,ID,Tile_name,LIVSM1,LIVSM2,mult_bin_labels,1,2,3,4,5,6,7,8,9,10,11,12,99
0,386,all_bands_merged_15_29.tif,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,0,0,0,0,0,0,0,0,0,0,0,0
1,140,all_bands_merged_08_25.tif,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,0,0,0,0,0,0,0,0,0,0,0,0
2,385,all_bands_merged_15_28.tif,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,0,0,0,0,0,0,0,0,0,0,0,0
3,311,all_bands_merged_13_28.tif,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,0,0,0,0,0,0,0,0,0,0,0,0
4,111,all_bands_merged_07_26.tif,1,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1,0,0,0,0,0,0,0,0,0,0,0,0


In [221]:
# Size of classes
labelfile_balanced_counted = labelfile_balanced.groupby('LIVSM1')['ID'].nunique()
labelfile_balanced_counted

LIVSM1
1     12
2     12
5     11
99    12
Name: ID, dtype: int64

## Homemade image data generator

In [222]:
def flip_vertical(image):
    return np.flipud(image)

def flip_horizontal(image):
    return np.fliplr(image)


def rotate_n_deg(image):
    # Rotates the image a random amount. Fills the space with a reflection of the image. Kva blir best her? Heller bruke rot90?
    rot = random.randrange(-180,180,30)
    rot_image = scipy.ndimage.interpolation.rotate(image, rot,axes=(1, 0), reshape=False, output=None, order=3, mode='reflect', cval=0.0, prefilter=True)

    return rot_image

#def rotate_180(image):
#    return np.rot90(image, k=2)

def random_image_flip_rotation(image):
    # This function rotates or flips the image randomly using one of five functions. 
    # This function can also do nothing to the image. 1/4 chanche.
    # Returns a image

    randint = np.random.randint(low=1, high=4)
    modified_image = 0
    
    if randint == 1:
        modified_image = flip_vertical(image)
    elif randint == 2:
        modified_image = flip_horizontal(image)

    elif randint == 3:
        modified_image = rotate_n_deg(image)

    else:
        modified_image = image
        
    return modified_image

A = np.array([[1, 2, 3], [4, 5, 6], [7,8,9]])
print(A)
print(flip_vertical(A))
print(flip_horizontal(A))
print(rotate_n_deg(A))
print(random_image_flip_rotation(A))

[[1 2 3]
 [4 5 6]
 [7 8 9]]
[[7 8 9]
 [4 5 6]
 [1 2 3]]
[[3 2 1]
 [6 5 4]
 [9 8 7]]
[[3 6 9]
 [2 5 8]
 [1 4 7]]
[[3 2 1]
 [6 5 4]
 [9 8 7]]


In [223]:
def get_input(path):
    
    img = imageio.imread(path)
    
    return(img)


def image_generator(image_path, label_file):
    # image_path: string where the images are placed.
    # label_file: csv file with labels and filenames.
    
    while True:
        batch_input = []
        batch_x = 0
        
        # Samples the file for a image, removes index so it can be used as loc[0]
        label_sample = label_file.sample().reset_index(drop=True)
        
        # Creates the file paths for the image
        image_sample_path = image_path + label_sample['Tile_name'].item()

        # Reads in each image as array, performs argumentation on each image
        input_raw = get_input(image_sample_path)
        input_permutated = random_image_flip_rotation(input_raw)
        
        # Add a new dimention to the array because Keras wants it this way?
        input_new_dim = np.expand_dims(input_permutated, axis=0)

        
        # Set a label by using the tile name. This line first finds the rowdata for 
        # a spesific tile and gives it out as a dataframe. Then the .iloc function 
        # finds the values of the colums 1 up to 99 and give them out as a list(list in a list). 
        label = label_sample.loc[0, ['1','2','3','4','5','6','7','8','9','10','11','12','99']].values
        
        # Place the labels inside a list in a list because Keras wants it this way??
        label = [[label]]

        yield(input_new_dim, label)

In [225]:
test_generated_images = image_generator(image_dir_path, labelfile_balanced)
generert = next(test_generated_images)
print(generert)
#print(next(test_generated_images))
print(np.shape(generert[0]))
print(generert[1])

(array([[[[7.1224053e-03, 7.1714274e-03, 6.6521158e-03, ...,
          0.0000000e+00, 0.0000000e+00, 1.3110821e-05],
         [6.9298497e-03, 8.0190385e-03, 7.2218417e-03, ...,
          0.0000000e+00, 0.0000000e+00, 1.3110821e-05],
         [8.1814639e-03, 7.6799942e-03, 8.5512009e-03, ...,
          7.6169010e-05, 3.9785362e-05, 0.0000000e+00],
         ...,
         [8.7966174e-03, 8.3507290e-03, 7.6553286e-03, ...,
          8.6783228e-05, 2.5322552e-05, 0.0000000e+00],
         [6.0452609e-03, 5.8857575e-03, 6.8950043e-03, ...,
          8.6783228e-05, 2.5322552e-05, 0.0000000e+00],
         [7.0037325e-03, 7.3904414e-03, 9.0437979e-03, ...,
          9.2565519e-05, 2.2261811e-04, 8.9161571e-05]],

        [[7.6876492e-03, 8.4293084e-03, 8.6015975e-03, ...,
          0.0000000e+00, 0.0000000e+00, 8.3537408e-05],
         [5.7936814e-03, 8.2197022e-03, 7.6892558e-03, ...,
          0.0000000e+00, 0.0000000e+00, 8.3537408e-05],
         [7.5824517e-03, 6.8531325e-03, 7.1922000e-03, 

## Keras model

In [292]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(50, 50, 474)))

model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(13, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 48, 48, 32)        136544    
_________________________________________________________________
global_average_pooling2d_13  (None, 32)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               8448      
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 13)                3341      
Total params: 148,333
Trainable params: 148,333
Non-trainable params: 0
_________________________________________________________________


In [296]:
model.fit_generator(
        image_generator(image_dir_path, labelfile_balanced),
        steps_per_epoch=10,
        epochs=5)

Epoch 1/5
10/10 [==============================] - 2s 215ms/step - loss: 0.3664 - acc: 0.8615
Epoch 2/5
10/10 [==============================] - 2s 230ms/step - loss: 0.3415 - acc: 0.9000
Epoch 3/5
10/10 [==============================] - 2s 173ms/step - loss: 0.3070 - acc: 0.8923
Epoch 4/5
10/10 [==============================] - 2s 195ms/step - loss: 0.3602 - acc: 0.8385
Epoch 5/5
10/10 [==============================] - 2s 161ms/step - loss: 0.3086 - acc: 0.8615


In [297]:
test_image_predict = labelfile_balanced.sample()
test_image_predict.head()

,ID,Tile_name,LIVSM1,LIVSM2,mult_bin_labels,1,2,3,4,5,6,7,8,9,10,11,12,99
24,375,all_bands_merged_15_18.tif,5,4,"[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,1,1,0,0,0,0,0,0,0,0


In [298]:
nn = model.predict_generator(
        image_generator(image_dir_path, test_image_predict),
        steps=1)
print(np.round(nn,2))

[[0.11 0.35 0.26 0.22 0.27 0.01 0.01 0.01 0.01 0.01 0.01 0.   0.07]]


In [282]:
sum(nn[0])

1.3441363036981784